# 라이브러리 불러오기

In [24]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
import os
import datetime

from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
from datetime import datetime, timedelta
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")

# 한글 깨지는거 방지하기
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
mpl.rcParams['axes.unicode_minus'] = False   
#그래프에서 마이너스 기호가 표시되도록 하는 설정

In [25]:
# 훈련 데이터를 읽어온다

train = pd.read_csv("../input/AFSNT.CSV", engine = "python", encoding ='CP949')
train.head()

,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP,ODP,FLO,FLT,REG,AOD,IRR,STT,ATT,DLY,DRR,CNL,CNR
0,2017,1,1,일,ARP3,ARP6,J,J1955,SEw3NzE4,D,N,10:05,10:32,N,NaN,N,NaN
1,2017,1,1,일,ARP3,ARP6,J,J1954,SEw3NzE4,A,N,9:30,9:31,N,NaN,N,NaN
2,2017,1,1,일,ARP3,ARP6,J,J1956,SEw3NzE4,A,N,12:45,13:03,N,NaN,N,NaN
3,2017,1,1,일,ARP3,ARP6,J,J1957,SEw3NzE4,D,N,13:25,14:09,Y,C02,N,NaN
4,2017,1,1,일,ARP3,ARP6,J,J1958,SEw3NzE4,A,N,16:10,16:31,N,NaN,N,NaN


In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987709 entries, 0 to 987708
Data columns (total 17 columns):
SDT_YY    987709 non-null int64
SDT_MM    987709 non-null int64
SDT_DD    987709 non-null int64
SDT_DY    987709 non-null object
ARP       987709 non-null object
ODP       987709 non-null object
FLO       987709 non-null object
FLT       987709 non-null object
REG       979446 non-null object
AOD       987709 non-null object
IRR       987709 non-null object
STT       987709 non-null object
ATT       987709 non-null object
DLY       987709 non-null object
DRR       118937 non-null object
CNL       987709 non-null object
CNR       8259 non-null object
dtypes: int64(3), object(14)
memory usage: 128.1+ MB


# 같은 항공편이 출도착으로 나뉘어 row로 존재하므로 이를 합친다.

In [27]:
# 출발하는 데이터/ 도착하는 데이터로 구분
trainA = train[train.AOD == "A"]
print(len(trainA))
trainD = train[train.AOD == "D"]
print(len(trainD))

493992
493717


In [28]:
# 년, 월, 일, 요일, 항공사, 항공편, 식별번호로 묶어서 데이터 행 합치기
all_df = pd.merge(left=trainD, right=trainA, how='inner',on=['SDT_YY','SDT_MM','SDT_DD', "SDT_DY", "FLO", "FLT", "REG"], sort=False)
all_df.head()

,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP_x,ODP_x,FLO,FLT,REG,AOD_x,IRR_x,STT_x,ATT_x,DLY_x,DRR_x,CNL_x,CNR_x,ARP_y,ODP_y,AOD_y,IRR_y,STT_y,ATT_y,DLY_y,DRR_y,CNL_y,CNR_y
0,2017,1,1,일,ARP3,ARP6,J,J1955,SEw3NzE4,D,N,10:05,10:32,N,NaN,N,NaN,ARP6,ARP3,A,N,11:10,11:18,N,NaN,N,NaN
1,2017,1,1,일,ARP3,ARP6,J,J1957,SEw3NzE4,D,N,13:25,14:09,Y,C02,N,NaN,ARP6,ARP3,A,N,14:30,14:56,N,NaN,N,NaN
2,2017,1,1,일,ARP3,ARP6,J,J1959,SEw3NTk5,D,N,16:45,17:21,Y,C02,N,NaN,ARP6,ARP3,A,N,17:50,18:07,N,NaN,N,NaN
3,2017,1,1,일,ARP3,ARP6,J,J1961,SEw3NTk5,D,N,20:35,20:52,N,NaN,N,NaN,ARP6,ARP3,A,N,21:40,21:39,N,NaN,N,NaN
4,2017,1,1,일,ARP1,ARP3,J,J1242,SEw3NzA2,D,N,20:25,20:36,N,NaN,N,NaN,ARP3,ARP1,A,N,21:30,21:27,N,NaN,N,NaN


In [29]:
all_df.shape

(492436, 27)

In [30]:
all_df.head()

,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP_x,ODP_x,FLO,FLT,REG,AOD_x,IRR_x,STT_x,ATT_x,DLY_x,DRR_x,CNL_x,CNR_x,ARP_y,ODP_y,AOD_y,IRR_y,STT_y,ATT_y,DLY_y,DRR_y,CNL_y,CNR_y
0,2017,1,1,일,ARP3,ARP6,J,J1955,SEw3NzE4,D,N,10:05,10:32,N,NaN,N,NaN,ARP6,ARP3,A,N,11:10,11:18,N,NaN,N,NaN
1,2017,1,1,일,ARP3,ARP6,J,J1957,SEw3NzE4,D,N,13:25,14:09,Y,C02,N,NaN,ARP6,ARP3,A,N,14:30,14:56,N,NaN,N,NaN
2,2017,1,1,일,ARP3,ARP6,J,J1959,SEw3NTk5,D,N,16:45,17:21,Y,C02,N,NaN,ARP6,ARP3,A,N,17:50,18:07,N,NaN,N,NaN
3,2017,1,1,일,ARP3,ARP6,J,J1961,SEw3NTk5,D,N,20:35,20:52,N,NaN,N,NaN,ARP6,ARP3,A,N,21:40,21:39,N,NaN,N,NaN
4,2017,1,1,일,ARP1,ARP3,J,J1242,SEw3NzA2,D,N,20:25,20:36,N,NaN,N,NaN,ARP3,ARP1,A,N,21:30,21:27,N,NaN,N,NaN


# 결항 데이터를 없앤다.

In [31]:
## 결항된 데이터 제거를 위해 결항 갯수 확인

print(len(all_df))
all_df_x = all_df[all_df.CNL_x == "Y"]
print(len(all_df_x))
all_df_y = all_df[all_df.CNL_y == "Y"]
print(len(all_df_y))

492436
4079
4079


In [32]:
## 결항 데이터 제거

all_df = all_df[all_df.CNL_x == "N"]
print(len(all_df))

# 확인
print(all_df[all_df.CNL_x =="N"].count())
print(all_df[all_df.CNL_y =="N"].count())


488357
SDT_YY    488357
SDT_MM    488357
SDT_DD    488357
SDT_DY    488357
ARP_x     488357
ODP_x     488357
FLO       488357
FLT       488357
REG       488357
AOD_x     488357
IRR_x     488357
STT_x     488357
ATT_x     488357
DLY_x     488357
DRR_x      85645
CNL_x     488357
CNR_x          0
ARP_y     488357
ODP_y     488357
AOD_y     488357
IRR_y     488357
STT_y     488357
ATT_y     488357
DLY_y     488357
DRR_y      32720
CNL_y     488357
CNR_y          0
dtype: int64
SDT_YY    488357
SDT_MM    488357
SDT_DD    488357
SDT_DY    488357
ARP_x     488357
ODP_x     488357
FLO       488357
FLT       488357
REG       488357
AOD_x     488357
IRR_x     488357
STT_x     488357
ATT_x     488357
DLY_x     488357
DRR_x      85645
CNL_x     488357
CNR_x          0
ARP_y     488357
ODP_y     488357
AOD_y     488357
IRR_y     488357
STT_y     488357
ATT_y     488357
DLY_y     488357
DRR_y      32720
CNL_y     488357
CNR_y          0
dtype: int64


In [33]:
all_df.shape

(488357, 27)

# 변수 제외 및 이름을 바꾼다.

In [34]:
# 필요없는 변수 제외하기
# 총 10개
# 겹치는 것 'ARP_y','ODP_y' = 공항, 상대공항 - 하나만 있으면 충분
# 필요없는 것 'AOD_x','AOD_y' = 출도착 여부 /'IRR_x','IRR_y' = 부정기편 여부 / 'CNL_x','CNR_x','CNL_y','CNR_y' = 결항관련 항목들
remove_variable = ['AOD_x','IRR_x','ARP_y','ODP_y','AOD_y','IRR_y','CNL_x','CNR_x','CNL_y','CNR_y']
all_df.drop(remove_variable, axis = 1, inplace = True)
all_df.head()

,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP_x,ODP_x,FLO,FLT,REG,STT_x,ATT_x,DLY_x,DRR_x,STT_y,ATT_y,DLY_y,DRR_y
0,2017,1,1,일,ARP3,ARP6,J,J1955,SEw3NzE4,10:05,10:32,N,NaN,11:10,11:18,N,NaN
1,2017,1,1,일,ARP3,ARP6,J,J1957,SEw3NzE4,13:25,14:09,Y,C02,14:30,14:56,N,NaN
2,2017,1,1,일,ARP3,ARP6,J,J1959,SEw3NTk5,16:45,17:21,Y,C02,17:50,18:07,N,NaN
3,2017,1,1,일,ARP3,ARP6,J,J1961,SEw3NTk5,20:35,20:52,N,NaN,21:40,21:39,N,NaN
4,2017,1,1,일,ARP1,ARP3,J,J1242,SEw3NzA2,20:25,20:36,N,NaN,21:30,21:27,N,NaN


In [35]:
## 변수명 및 값들 한국어로 바꾸기

print('Dataframe dimensions:', all_df.shape)
all_df.columns = ['년','월','일', '요일', '공항', '상대공항', '항공사', '항공편', '식별번호', '예정출발시간',
             '실제출발시간','출발편지연여부', '출발편지연사유', '예정도착시간', '실제도착시간', '도착편지연여부', '도착편지연사유', ]
#recollist = ['년','월','일', '요일', '공항', '출도착', '상대공항', '항공사', '항공편', '식별번호', 
#              '계획', '실제', '지연', '지연사유', '결항', '결항사유', '부정기편']
#all_df = all_df[recollist]
transairport = {'ARP1':'김포', 'ARP2':'김해', 'ARP3': '제주', 'ARP4':'대구', 'ARP5': '울산',
           'ARP6':'청주', 'ARP7':'무안', 'ARP8':'광주', 'ARP9':'여수', 'ARP10':'양양',
           'ARP11':'포항', 'ARP12':'사천', 'ARP13':'군산', 'ARP14':'원주', 'ARP15':'인천'}
all_df['공항'] = all_df['공항'].apply(lambda x: transairport[x])
all_df['상대공항'] = all_df['상대공항'].apply(lambda x: transairport[x])
#train_inner["출도착"] = train_inner["출도착"].apply(lambda x: "출발" if x=="D" else x)
#train_inner["출도착"] = train_inner["출도착"].apply(lambda x: "도착" if x=="A" else x)
#df = df.iloc[df['항공사']!= ['C', 'D', 'E', 'G', 'K'],]
transairline = {'A':'아시아나', 'B':'에어부산', 'C': '전일본항공ANA', 'D':'에어서울', 'E': '불명',
           'F':'이스타', 'G':'일본항공', 'H':'제주항공', 'I':'진에어', 'J':'대한항공',
           'K':'타이완', 'L':'티웨이', 'M':'신규'}
all_df['항공사'] = all_df['항공사'].apply(lambda x: transairline[x])

Dataframe dimensions: (488357, 17)


In [36]:
all_df.head()

,년,월,일,요일,공항,상대공항,항공사,항공편,식별번호,예정출발시간,실제출발시간,출발편지연여부,출발편지연사유,예정도착시간,실제도착시간,도착편지연여부,도착편지연사유
0,2017,1,1,일,제주,청주,대한항공,J1955,SEw3NzE4,10:05,10:32,N,NaN,11:10,11:18,N,NaN
1,2017,1,1,일,제주,청주,대한항공,J1957,SEw3NzE4,13:25,14:09,Y,C02,14:30,14:56,N,NaN
2,2017,1,1,일,제주,청주,대한항공,J1959,SEw3NTk5,16:45,17:21,Y,C02,17:50,18:07,N,NaN
3,2017,1,1,일,제주,청주,대한항공,J1961,SEw3NTk5,20:35,20:52,N,NaN,21:40,21:39,N,NaN
4,2017,1,1,일,김포,제주,대한항공,J1242,SEw3NzA2,20:25,20:36,N,NaN,21:30,21:27,N,NaN


# 다음날로 넘어가는 항공편들 x_차이를 구하기 위한 작업들

In [37]:
all_df['예정출발시간'] = pd.to_datetime(all_df['예정출발시간'])
all_df['실제출발시간'] = pd.to_datetime(all_df['실제출발시간'])
all_df['예정도착시간'] = pd.to_datetime(all_df['예정도착시간'])
all_df['실제도착시간'] = pd.to_datetime(all_df['실제도착시간'])

In [38]:
all_df['time_exp_dep'] = all_df['예정출발시간'].apply(lambda x : x.time().hour)
all_df['time_act_dep'] = all_df['실제출발시간'].apply(lambda x : x.time().hour)
all_df['time_exp_ari'] = all_df['예정도착시간'].apply(lambda x : x.time().hour)
all_df['time_act_ari'] = all_df['실제도착시간'].apply(lambda x : x.time().hour)

def nextday_time_mapping_dep(row):
  if (row['time_exp_dep'] >= 20) and ((row['time_act_dep'] >=0) and (row['time_act_dep'] <= 5)):
    return row['실제출발시간'] + timedelta(days = +1)
  else:
    return row['실제출발시간']

def nextday_time_mapping_ari(row):
  if (row['time_exp_ari'] >= 20.0) and ((row['time_act_ari'] >=0) and (row['time_act_ari'] <= 5)):
    return row['실제도착시간'] + timedelta(days = +1)
  else: 
    return row['실제도착시간']

In [39]:
print(pd.to_datetime('today').day)

today_day = pd.to_datetime('today').day

all_df['실제출발시간'] = all_df.apply(nextday_time_mapping_dep, axis = 1)
all_df['실제도착시간'] = all_df.apply(nextday_time_mapping_ari, axis = 1)


7


In [40]:
def check_dep(row):
#     print(row['실제출발시간'].day)
#     print(row)
    if row['실제출발시간'].day == today_day+1:
        print(row['실제출발시간'].day)

def check_ari(row):
#     print(row['실제출발시간'].day)
#     print(row)
    if row['실제도착시간'].day == today_day+1:
        print(row['실제도착시간'].day)

all_df.apply(check_dep, axis = 1)
all_df.apply(check_ari, axis = 1)


8
8
8
8
8
8
8
8
8
8
8
8


0         None
1         None
2         None
3         None
4         None
5         None
6         None
7         None
8         None
9         None
10        None
11        None
12        None
13        None
14        None
15        None
16        None
17        None
18        None
19        None
20        None
21        None
22        None
23        None
24        None
25        None
26        None
27        None
28        None
29        None
          ... 
492406    None
492407    None
492408    None
492409    None
492410    None
492411    None
492412    None
492413    None
492414    None
492415    None
492416    None
492417    None
492418    None
492419    None
492420    None
492421    None
492422    None
492423    None
492424    None
492425    None
492426    None
492427    None
492428    None
492429    None
492430    None
492431    None
492432    None
492433    None
492434    None
492435    None
Length: 488357, dtype: object

In [41]:
## 지역시간x , 지연시간 y , 계획 비행 시간 차이, 실제 비행 시간 차이 구하기 
all_df["x_차이"] = all_df["실제출발시간"] - all_df["예정출발시간"]
all_df["y_차이"] = all_df["실제도착시간"] - all_df["예정도착시간"]
all_df["plan_차이"] = all_df["예정도착시간"] - all_df["예정출발시간"]
all_df["true_차이"] = all_df["실제도착시간"] - all_df["실제출발시간"]
## - 값을 따로 넣어주기 위해 양수, 음수 값 변환
all_df["x_차이_minus"] = all_df["예정출발시간"] - all_df["실제출발시간"]
all_df["y_차이_minus"] = all_df["예정도착시간"] - all_df["실제도착시간"]
all_df["plan_차이_minus"] = all_df["예정출발시간"] - all_df["예정도착시간"]
all_df["true_차이_minus"] = all_df["실제출발시간"] - all_df["실제도착시간"]
## 더 빨리 도착한 값은 -를 붙여서 변환, 지연된 값은 그대로 저장
all_df['x_차이'][all_df['x_차이'].astype('timedelta64[m]') > 0 ] =   all_df['x_차이'].astype('timedelta64[m]') 
all_df['x_차이'][all_df['x_차이_minus'].astype('timedelta64[m]') > 0 ] = 0 - all_df['x_차이_minus'].astype('timedelta64[m]') 
all_df['y_차이'][all_df['y_차이'].astype('timedelta64[m]') > 0 ] =   all_df['y_차이'].astype('timedelta64[m]') 
all_df['y_차이'][all_df['y_차이_minus'].astype('timedelta64[m]') > 0 ] = 0 - all_df['y_차이_minus'].astype('timedelta64[m]') 
all_df['plan_차이'][all_df['plan_차이'].astype('timedelta64[m]') > 0 ] =   all_df['plan_차이'].astype('timedelta64[m]') 
all_df['plan_차이'][all_df['plan_차이_minus'].astype('timedelta64[m]') > 0 ] = 0 - all_df['plan_차이_minus'].astype('timedelta64[m]') 
all_df['true_차이'][all_df['true_차이'].astype('timedelta64[m]') > 0 ] =   all_df['true_차이'].astype('timedelta64[m]') 
all_df['true_차이'][all_df['true_차이_minus'].astype('timedelta64[m]') > 0 ] = 0 - all_df['true_차이_minus'].astype('timedelta64[m]') 
## minus 칼럼 제거 
all_df = all_df.drop(["x_차이_minus", "y_차이_minus", "plan_차이_minus","true_차이_minus"], axis = 1)


# x_차이 튀는 값들 제거

In [42]:
idx_1 = all_df[((all_df['x_차이'] > 30) | (all_df['x_차이'] < -30.0)) & (all_df['출발편지연여부'] == 'N')].index
all_df = all_df.drop(idx_1,axis = 0)
idx_2 = all_df[((all_df['x_차이'] <= 30) & (all_df['x_차이'] >= -30.0)) & (all_df['출발편지연여부'] == 'Y')].index
all_df = all_df.drop(idx_2,axis = 0)
idx_3 = all_df[((all_df['y_차이'] > 30) | (all_df['y_차이'] < -30.0)) & (all_df['도착편지연여부'] == 'N')].index
all_df = all_df.drop(idx_3,axis = 0)
idx_4 = all_df[((all_df['y_차이'] <= 30) & (all_df['y_차이'] >= -30.0)) & (all_df['도착편지연여부'] == 'Y')].index
all_df = all_df.drop(idx_4,axis = 0)
all_df.shape

(487695, 25)

# 비행시간 차이 feature 만들기

In [43]:
all_df["비행시간차이"] = all_df["plan_차이"] - all_df["true_차이"]

all_df.head()

,년,월,일,요일,공항,상대공항,항공사,항공편,식별번호,예정출발시간,실제출발시간,출발편지연여부,출발편지연사유,예정도착시간,실제도착시간,도착편지연여부,도착편지연사유,time_exp_dep,time_act_dep,time_exp_ari,time_act_ari,x_차이,y_차이,plan_차이,true_차이,비행시간차이
0,2017,1,1,일,제주,청주,대한항공,J1955,SEw3NzE4,2019-09-07 10:05:00,2019-09-07 10:32:00,N,NaN,2019-09-07 11:10:00,2019-09-07 11:18:00,N,NaN,10,10,11,11,27.0,8.0,65.0,46.0,19.0
1,2017,1,1,일,제주,청주,대한항공,J1957,SEw3NzE4,2019-09-07 13:25:00,2019-09-07 14:09:00,Y,C02,2019-09-07 14:30:00,2019-09-07 14:56:00,N,NaN,13,14,14,14,44.0,26.0,65.0,47.0,18.0
2,2017,1,1,일,제주,청주,대한항공,J1959,SEw3NTk5,2019-09-07 16:45:00,2019-09-07 17:21:00,Y,C02,2019-09-07 17:50:00,2019-09-07 18:07:00,N,NaN,16,17,17,18,36.0,17.0,65.0,46.0,19.0
3,2017,1,1,일,제주,청주,대한항공,J1961,SEw3NTk5,2019-09-07 20:35:00,2019-09-07 20:52:00,N,NaN,2019-09-07 21:40:00,2019-09-07 21:39:00,N,NaN,20,20,21,21,17.0,-1.0,65.0,47.0,18.0
4,2017,1,1,일,김포,제주,대한항공,J1242,SEw3NzA2,2019-09-07 20:25:00,2019-09-07 20:36:00,N,NaN,2019-09-07 21:30:00,2019-09-07 21:27:00,N,NaN,20,20,21,21,11.0,-3.0,65.0,51.0,14.0


In [44]:
all_df.to_csv('all_df.csv', encoding = "CP949")